In [1]:
%pylab inline
from itertools import permutations, combinations_with_replacement

Populating the interactive namespace from numpy and matplotlib


In [2]:
def choose(n, k):
    if n < k:
        return 0
    
    else:
        num = math.factorial(n)
        den = math.factorial(k)*math.factorial(n - k)
    return int(num/den)

In [3]:
def gen_pi(K = 4, v = 20, acorn = 1234):
    np.random.seed(acorn)
    #Taken from Bijan
    
    while True:
        pi_0 = np.random.uniform(size=K) # Generate K Uniform(0,1) random variables
        pisum = np.sum(pi_0) # Calculate the sum of the K generated Uniform(0,1) random variables
        pi = [i/pisum for i in pi_0] # Normalize so that sum(pi) = 1
        Nv = [int(round(i*v)) for i in pi] # Round so that each block has an integer-valued amount of vertices
        if not 0 in Nv and np.sum(Nv)==v: # Make sure no block has 0 vertices and the sum of all vertices is correct
            break
    return pi, Nv # returns the vertex assignment distribution and the number of vertices in each block

In [4]:
def gen_Lambda(K, acorn=1234, ones_ = False):
    if ones_ == True:
        return np.ones(shape = (K, K))
    
    np.random.seed(acorn)
    Lambda = zeros(shape = (K, K)) # K x K matrix to store adjacency probabilities
    for i in range(K): # for each block
        for j in range(i, K): # for each combination (with replacement)
            Lambda[i, j] = np.random.uniform() # generate a Uniform(0,1) random variable
            Lambda[j, i] = Lambda[i, j] # Lambda is symmetric
                   
    return Lambda # returns a K x K, symmetric matrix where Lambda[i,j] in (0, 1)

In [5]:
def adj_matrix(n, pi, Lambda, acorn = 1234):
    np.random.seed(acorn)
    n = int(n) # Just in case!
    A = np.zeros(shape = (n, n)) # n x n adjcacency matrix
    K = len(pi) # extract the number of blocks in the SBM
    
    i = 0 # start at block indexed with 0
    while i < K: # while the block number is less than the total number of blocks
        for k in range(int(round(n*(sum(pi[:i])))), int(round(n*(sum(pi[:i + 1]))))): # for all vertices in block i
            c = i # start at block i
            while c < K: # while the block number is less than the total number of blocks
                for j in range(int(round(n*(sum(pi[:c])))), int(round(n*(sum(pi[:c + 1]))))): # for all vertices in block c
                    A[k, j] = np.random.binomial(1, Lambda[i, c]) # generates and assigns an edge based on block membership
                    A[j, k] = A[k, j] # A is symmetric
                c += 1
            A[k,k] = 0 # A is hollow
        i += 1
        
    return A # returns an n x n, symmetric and hollow matrix where A[i,j] in {0, 1}

In [6]:
def gen_seeds(Nv, seed_ratio, acorn = 1234):
    np.random.seed(acorn)
    
    K = len(Nv)
    
    num_seeds = [int(round(seed_ratio*Nv)) for i in range(K)]
    seeds = [[] for i in m]
    for i in range(K):
        for j in range(num_seeds[i]):
            index = np.random.randint(j)

In [97]:
def gen_e(A, perm, seeds):
    K = len(perm)
    e = np.zeros(shape = (K, K)) # create an ((K choose 2) + K) x ((K choose 2) + K)
        # to store e[k,l]
    
    #V = seeds[0] + perm[0]
    V = [seeds[i] + perm[i] for i in range(K)]
    #print(V)
        
    #e_test = np.zeros(shape = (choose(len(V), 2) + K, choose(len(V), 2) + K))
    
    for i in range(K):
        for j in range(i + 1, K):
            #print(V)
            for v in V[i]:
                for s in V[j]:
                    e[i,j] += A[v, s]
    
    for i in range(K):
        for j in range(len(V[i])):
            for k in range(j + 1, len(V[i])):
                e[i, i] += A[j, k]
        
    return e #, e_test

In [98]:
def gen_c(Nv, e):
    K = len(Nv) # number of blocks
    c = np.zeros(shape = (K, K)) # create an ((K choose 2) + K) x ((K choose 2) + K)
    #print(Nv)
    
    for i in range(K): # for each block..
        for j in range(i, K): # for every other block..
            if i == j: # diagonal entries
                c[i,j] = choose(Nv[i], 2) - e[i,i] # Max possible edges minus realized edges
            else:
                c[i,j] = (Nv[i] * Nv[j]) - e[i,j] # Max possible edges minus realized edges
            
    return c

In [105]:
def gen_denom(V, Lambda, e, c):
    prod = 1
    for i in range(K):
        for j in range(i, K):
            prod *= Lambda[i,j]**e[i,j] * (1 - Lambda[i,j])**c[i,j]
            
    return prod

In [106]:
K = 3 # number of blocks 
v = 11 # number of vertices
v_list = list(arange(v))
labeled_ratio = 0.25 # ratio of labeled vertices
m = int(round(labeled_ratio*v)) # number of labeled vertices
n = int(v - m) # number of unlabeled vertices

pi, Nv = gen_pi(K, v, acorn = 123) # generate vertex distribution, total number of vertices in each block
mv = [int(round(labeled_ratio*Nv[i])) for i in range(K)] # generate the number of labeled vertices in each block
seeds = [[] for i in range(K)] # generate a list to store seed indices

nv = [int(round((1 - labeled_ratio)*Nv[i])) for i in range(K)] # generate the number of unlabeled vertices in each block
L = gen_Lambda(4, acorn = 3, ones_ = True) # generate adjacency probability matrix
               
A = adj_matrix(v, pi, L, acorn = 100) # generate adjacency matrix

""" Use when permutation matrix has identity in top left """
#count = 0 # first seed is indexed at 0
#for i in range(K): # for all blocks.. 
#    for k in range(mv[i]): # for the number of seeds in each block..
#        seeds[i].append(count) # append the count to the list of seeds
#        count += 1 # update 
        
""" Use when adjacency matrix is not permuted """
for i in range(K): # for all blocks.. 
    for k in range(mv[i]): # for the number of seeds in each block..
        seeds[i].append(int(i + k + sum(Nv[:i]))) # append the count to the list of seeds

all_seeds = []
for i in range(K):
    for j in range(len(seeds[i])):
        all_seeds.append(seeds[i][j])

unlabeled = [item for item in v_list if item not in all_seeds]

for p in permutations(unlabeled, n): # find maximally likely permutation
    V = [list(p[int(sum(nv[:i])):int(sum(nv[:i + 1]))]) for i in range(K)]
    e = gen_e(A, V, seeds)
    c = gen_c(Nv, e)
    denom += gen_denom(V, Lamb, e, c)
    break

NameError: name 'Lambda' is not defined

In [ ]:


for p in permutations(arange(m, n + m), n):
    V = [p[int(sum(n_k[:i])):int(sum(n_k[:i + 1]))] for i in range(K)]
    
    